In [2]:
import pandas as pd

In [59]:
df_peru= pd.read_csv('Peru.csv')
df_argentina = pd.read_csv('Argentina.csv')
df_colombia = pd.read_csv('Colombia.csv')

# Transformacion Peru

In [44]:
df_peru.head()

,location_key,date,country_code,country_name,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,population,population_male,...,nurses_per_1000,physicians_per_1000,population_largest_city,area_rural_sq_km,area_urban_sq_km,life_expectancy,adult_male_mortality_rate,adult_female_mortality_rate,pollution_mortality_rate,comorbidity_mortality_rate
0,PE,2020-07-21,PE,Peru,5313.0,601.0,384281.0,50781.0,29381884,14450757,...,2.4398,1.3048,10554712.0,1256339.0,16425.0,76.516,146.37,84.815,63.9,12.6
1,PE,2020-07-22,PE,Peru,6328.0,617.0,390609.0,51398.0,29381884,14450757,...,2.4398,1.3048,10554712.0,1256339.0,16425.0,76.516,146.37,84.815,63.9,12.6
2,PE,2020-07-23,PE,Peru,5871.0,623.0,396480.0,52021.0,29381884,14450757,...,2.4398,1.3048,10554712.0,1256339.0,16425.0,76.516,146.37,84.815,63.9,12.6
3,PE,2020-07-24,PE,Peru,6261.0,587.0,402741.0,52608.0,29381884,14450757,...,2.4398,1.3048,10554712.0,1256339.0,16425.0,76.516,146.37,84.815,63.9,12.6
4,PE,2020-07-25,PE,Peru,5875.0,616.0,408616.0,53224.0,29381884,14450757,...,2.4398,1.3048,10554712.0,1256339.0,16425.0,76.516,146.37,84.815,63.9,12.6


In [60]:
columns_to_drop = [
    'cumulative_confirmed', 
    'cumulative_deceased', 
    'rainfall_mm', 
    'new_recovered',
    'cumulative_recovered',
    'population_rural',
    'population_urban', 
    'minimum_temperature_celsius', 
    'maximum_temperature_celsius'
]

# Función para eliminar columnas
def process_dataframe(df, start_date, end_date, columns=columns_to_drop):

    # Verificar si el DataFrame es None
    if df is None:
        raise ValueError("El DataFrame proporcionado es None")

    # Ordenar por fecha
    df = df.sort_values(by='date')

    # Filtrar por rango de fechas
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

    # Eliminar las columnas especificadas
    df = df.drop(columns=columns, errors='ignore')

    # Imputación con la media para columnas específicas
    df['average_temperature_celsius'] = df['average_temperature_celsius'].fillna(df['average_temperature_celsius'].mean())
    df['relative_humidity'] = df['relative_humidity'].fillna(df['relative_humidity'].mean())

    # Calcular la mediana de la columna 'new_confirmed' y 'new_deceased'
    median_new_confirmed = df['new_confirmed'].median()
    median_new_deceased = df['new_deceased'].median()

    # Fechas específicas para imputar
    dates_to_impute = ['2022-05-06', '2022-06-01']

    # Reemplazar valores cero por la mediana en las fechas específicas
    for date in dates_to_impute:
        df.loc[(df['date'] == date) & (df['new_confirmed'] == 0), 'new_confirmed'] = median_new_confirmed
        df.loc[(df['date'] == date) & (df['new_deceased'] == 0), 'new_deceased'] = median_new_deceased

    fecha_limite = '2021-02-09'
    df.loc[df['date'] < fecha_limite] = df.loc[df['date'] < fecha_limite].fillna(0)
    df.isna().sum()

    # Calcular columna de vacunas administradas por día
    df['vaccine_doses_administered_for_day'] = df['cumulative_vaccine_doses_administered'].diff().fillna(0)

    # Encontrar el máximo de vacunas administradas en un día
    max_vaccine_doses = df['vaccine_doses_administered_for_day'].max()

    # Reorganizar las columnas
    df.drop(columns='cumulative_vaccine_doses_administered', inplace=True)
    df.insert(6, 'vaccine_doses_administered_for_day', df.pop('vaccine_doses_administered_for_day'))

    # Crear la columna del acumulado de vacunas diarias
    df['cumulative_vaccine_doses_administered'] = df['vaccine_doses_administered_for_day'].cumsum()
    df.insert(7, 'cumulative_vaccine_doses_administered', df.pop('cumulative_vaccine_doses_administered'))

    # Calcular el rango intercuartílico (IQR)
    Q1 = df['vaccine_doses_administered_for_day'].quantile(0.25)
    Q3 = df['vaccine_doses_administered_for_day'].quantile(0.75)
    IQR = Q3 - Q1

    # Definir los límites para considerar outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filtrar el DataFrame para mantener solo los valores dentro de los límites
    df_filtered = df[(df['vaccine_doses_administered_for_day'] >= lower_bound) & (df['vaccine_doses_administered_for_day'] <= upper_bound)]

    # Calcular el promedio entre los límites
    avg_vaccine = (upper_bound + lower_bound) / 2

    # Reemplazar los valores por encima del valor máximo con el promedio
    df['vaccine_doses_administered_for_day'] = df['vaccine_doses_administered_for_day'].apply(
        lambda x: avg_vaccine if x > upper_bound else x
    )
    
    return df

In [61]:
# Definir los rangos de fechas para cada país
start_date_peru = '2020-03-05'
end_date_peru = '2022-08-31'

# Aplicar la función a distintos DataFrames
df_peru = process_dataframe(df_peru, start_date_peru, end_date_peru)

In [ ]:
start_date_argentina = '2020-01-01'
end_date_argentina = '2022-08-31'

# Aplicar la función a distintos DataFrames
df_peru = process_dataframe(df_argentina, start_date_argentina, end_date_argentina)


# Transformaciones de Argentina

In [ ]:

columns_to_drop = [
    'cumulative_confirmed', 
    'cumulative_deceased', 
    'rainfall_mm', 
    'new_recovered',
    'cumulative_recovered',
    'population_rural',
    'population_urban', 
    'minimum_temperature_celsius', 
    'maximum_temperature_celsius'
]

# Función para eliminar columnas
def process_dataframe(df, start_date, end_date, columns=columns_to_drop):

    # Verificar si el DataFrame es None
    if df is None:
        raise ValueError("El DataFrame proporcionado es None")

    # Ordenar por fecha
    df = df.sort_values(by='date')

    # Filtrar por rango de fechas
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    
    # Eliminar las columnas especificadas
    df.drop(columns=columns, inplace=True, errors='ignore')
    
    # Imputación con la media para columnas específicas
    df['average_temperature_celsius'].fillna(df['average_temperature_celsius'].mean(), inplace=True)
    df['relative_humidity'].fillna(df['relative_humidity'].mean(), inplace=True)
    
    # Rellenar con 0 donde la fecha sea menor a '2021-02-17'
    fecha_limite = '2021-02-17'
    df_co.loc[df['date'] < fecha_limite] = df_co.loc[df['date'] < fecha_limite].fillna(0)
    
    # Calcular columna de vacunas administradas por día
    df['vaccine_doses_administered_for_day'] = df['cumulative_vaccine_doses_administered'].diff().fillna(0)
    
    # Reorganizar las columnas
    df.drop(columns='cumulative_vaccine_doses_administered', inplace=True)
    df.insert(6, 'vaccine_doses_administered_for_day', df.pop('vaccine_doses_administered_for_day'))
    
    # Crear la columna del acumulado de vacunas diarias
    df['cumulative_vaccine_doses_administered'] = df['vaccine_doses_administered_for_day'].cumsum()
    df.insert(7, 'cumulative_vaccine_doses_administered', df.pop('cumulative_vaccine_doses_administered'))
    
    # Calcular el rango intercuartílico (IQR)
    Q1 = df['vaccine_doses_administered_for_day'].quantile(0.25)
    Q3 = df['vaccine_doses_administered_for_day'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Definir los límites para considerar outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filtrar el DataFrame para mantener solo los valores dentro de los límites
    df_filtered = df[(df['vaccine_doses_administered_for_day'] >= lower_bound) & (df['vaccine_doses_administered_for_day'] <= upper_bound)]
    
    # Calcular el promedio entre los límites
    avg_vaccine = df_filtered['vaccine_doses_administered_for_day'].mean()
    
    # Reemplazar los valores por encima del valor máximo con el promedio
    df['vaccine_doses_administered_for_day'] = df['vaccine_doses_administered_for_day'].apply(
        lambda x: avg_vaccine if x > upper_bound else x
    )
    
    return df

In [ ]:
# Definir los rangos de fechas para cada país
start_date_argentina = '2020-01-01'
end_date_argentina = '2022-08-31'

# Aplicar la función a distintos DataFrames
df_argentina = process_dataframe(df_argentina, start_date_peru, end_date_argentina)

# Transformacion Colombia

In [ ]:
columns_to_drop = [
    'cumulative_confirmed', 
    'cumulative_deceased', 
    'rainfall_mm', 
    'new_recovered',
    'cumulative_recovered',
    'population_rural',
    'population_urban', 
    'minimum_temperature_celsius', 
    'maximum_temperature_celsius'
]

# Función para eliminar columnas
def process_dataframe(df, start_date, end_date, columns=columns_to_drop):

    # Verificar si el DataFrame es None
    if df is None:
        raise ValueError("El DataFrame proporcionado es None")

    # Ordenar por fecha
    df = df.sort_values(by='date')

    # Filtrar por rango de fechas
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    
    # Eliminar las columnas especificadas
    df.drop(columns=columns, inplace=True, errors='ignore')
    
    # Imputación con la media para columnas específicas
    df['average_temperature_celsius'].fillna(df['average_temperature_celsius'].mean(), inplace=True)
    df['relative_humidity'].fillna(df['relative_humidity'].mean(), inplace=True)
    
    # Rellenar con 0 donde la fecha sea menor a '2021-02-17'
    fecha_limite = '2021-02-17'
    df.loc[df['date'] < fecha_limite] = df.loc[df['date'] < fecha_limite].fillna(0)
    
    # Calcular columna de vacunas administradas por día
    df['vaccine_doses_administered_for_day'] = df['cumulative_vaccine_doses_administered'].diff().fillna(0)
    
    # Reorganizar las columnas
    df.drop(columns='cumulative_vaccine_doses_administered', inplace=True)
    df.insert(6, 'vaccine_doses_administered_for_day', df.pop('vaccine_doses_administered_for_day'))
    
    # Crear la columna del acumulado de vacunas diarias
    df['cumulative_vaccine_doses_administered'] = df['vaccine_doses_administered_for_day'].cumsum()
    df.insert(7, 'cumulative_vaccine_doses_administered', df.pop('cumulative_vaccine_doses_administered'))
    
    # Calcular el rango intercuartílico (IQR)
    Q1 = df['vaccine_doses_administered_for_day'].quantile(0.25)
    Q3 = df['vaccine_doses_administered_for_day'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Definir los límites para considerar outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filtrar el DataFrame para mantener solo los valores dentro de los límites
    df_filtered = df[(df['vaccine_doses_administered_for_day'] >= lower_bound) & (df['vaccine_doses_administered_for_day'] <= upper_bound)]
    
    # Calcular el promedio entre los límites
    avg_vaccine = df_filtered['vaccine_doses_administered_for_day'].mean()
    
    # Reemplazar los valores por encima del valor máximo con el promedio
    df['vaccine_doses_administered_for_day'] = df['vaccine_doses_administered_for_day'].apply(
        lambda x: avg_vaccine if x > upper_bound else x
    )
    
    return df

In [ ]:
start_date_colombia = '2020-01-01'
end_date_colombia = '2022-08-31'

# Aplicar la función a distintos DataFrames
df_colombia = process_dataframe(df_colombia, start_date_colombia, end_date_colombia)

# Transformacion Brasil

In [ ]:
columns_to_drop = [
    'cumulative_confirmed', 
    'cumulative_deceased', 
    'rainfall_mm', 
    'new_recovered',
    'cumulative_recovered',
    'population_rural',
    'population_urban', 
    'minimum_temperature_celsius', 
    'maximum_temperature_celsius'
]

# Función para eliminar columnas
def process_dataframe(df, start_date, end_date, columns=columns_to_drop):

    # Verificar si el DataFrame es None
    if df is None:
        raise ValueError("El DataFrame proporcionado es None")

    # Ordenar por fecha
    df = df.sort_values(by='date')

    # Filtrar por rango de fechas
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    
    # Eliminar las columnas especificadas
    df.drop(columns=columns, inplace=True, errors='ignore')
    
    # Imputación con la media para columnas específicas
    df['average_temperature_celsius'].fillna(df['average_temperature_celsius'].mean(), inplace=True)
    df['relative_humidity'].fillna(df['relative_humidity'].mean(), inplace=True)
    
    # Rellenar con 0 donde la fecha sea menor a '2021-02-17'
    fecha_limite = '2021-02-17'
    df.loc[df['date'] < fecha_limite] = df.loc[df['date'] < fecha_limite].fillna(0)
    
    # Calcular columna de vacunas administradas por día
    df['vaccine_doses_administered_for_day'] = df['cumulative_vaccine_doses_administered'].diff().fillna(0)
    
    # Reorganizar las columnas
    df.drop(columns='cumulative_vaccine_doses_administered', inplace=True)
    df.insert(6, 'vaccine_doses_administered_for_day', df.pop('vaccine_doses_administered_for_day'))
    
    # Crear la columna del acumulado de vacunas diarias
    df['cumulative_vaccine_doses_administered'] = df['vaccine_doses_administered_for_day'].cumsum()
    df.insert(7, 'cumulative_vaccine_doses_administered', df.pop('cumulative_vaccine_doses_administered'))
    
    # Calcular el rango intercuartílico (IQR)
    Q1 = df['vaccine_doses_administered_for_day'].quantile(0.25)
    Q3 = df['vaccine_doses_administered_for_day'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Definir los límites para considerar outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filtrar el DataFrame para mantener solo los valores dentro de los límites
    df_filtered = df[(df['vaccine_doses_administered_for_day'] >= lower_bound) & (df['vaccine_doses_administered_for_day'] <= upper_bound)]
    
    # Calcular el promedio entre los límites
    avg_vaccine = df_filtered['vaccine_doses_administered_for_day'].mean()
    
    # Reemplazar los valores por encima del valor máximo con el promedio
    df['vaccine_doses_administered_for_day'] = df['vaccine_doses_administered_for_day'].apply(
        lambda x: avg_vaccine if x > upper_bound else x
    )
    
    return df